In [1]:
import ebooklib
from ebooklib import epub

def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

from bs4 import BeautifulSoup
blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', ]
# there may be more elements you don't want, such as "style", etc.

def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [2]:
harrypotter=epub2text('Harry Potter.epub')
davinci=epub2text('Da Vinci Code.epub')
clouds=epub2text('Head in the Clouds.epub')
cosmos=epub2text("Cosmos.epub")

In [3]:
#harrypotter

In [4]:
#davinci

In [5]:
#clouds

In [6]:
#cosmos

In [20]:
import pandas as pd
import re

harrypotter_text = ''
for o in harrypotter:
    harrypotter_text = harrypotter_text + o
    
harrypotter_lines = harrypotter_text.splitlines()
harrypotter_df = pd.DataFrame({
    "content": harrypotter_lines,
    "line": list(range(len(harrypotter_lines)))
})
harrypotter_df

,content,line
0,,0
1,,1
2,,2
3,,3
4,HP 4 - Harry Potter and The Goblet of Fire,4
...,...,...
6764,"Har­ry winked at them, turned to Un­cle...",6764
6765,"As Ha­grid had said, what would come, w...",6765
6766,,6766
6767,,6767


In [8]:
davinci_text = ''
for o in davinci:
    davinci_text = davinci_text + o
    
davinci_lines = davinci_text.splitlines()
davinci_df = pd.DataFrame({
    "content": davinci_lines,
    "line": list(range(len(davinci_lines)))
})
davinci_df

,content,line
0,,0
1,The Da Vinci Code,1
2,Dan Brown,2
3,,3
4,FOR BLYTHE... AGAIN. MORE THAN EVER.,4
...,...,...
5887,,5887
5888,,5888
5889,,5889
5890,,5890


In [9]:
clouds_text = ''
for o in clouds:
    clouds_text = clouds_text + o
    
clouds_lines = clouds_text.splitlines()
clouds_df = pd.DataFrame({
    "content": clouds_lines,
    "line": list(range(len(clouds_lines)))
})
clouds_df

,content,line
0,,0
1,Head,1
2,IN THE,2
3,Clouds,3
4,,4
...,...,...
266,,266
267,,267
268,,268
269,,269


In [10]:
cosmos_text = ''
for o in cosmos:
    cosmos_text = cosmos_text + o
    
cosmos_lines = cosmos_text.splitlines()
cosmos_df = pd.DataFrame({
    "content": cosmos_lines,
    "line": list(range(len(cosmos_lines)))
})
cosmos_df

,content,line
0,ACKNOWLEDGMENTS,0
1,"Besides those thanked in the introduction, I ...",1
2,,2
3,By Carl Sagan Published by The Random House P...,3
4,,4
...,...,...
1703,,1703
1704,,1704
1705,,1705
1706,,1706


In [21]:
def line_is_chapter_harrypotter(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("^CHAPTER ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN|EIGHT|NINE|TEN|ELEVEN|TWELVE|.*TEEN|TWENTY\-?[A-Z]*|THIRTY\-?[A-Z]*", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
harrypotter_df = harrypotter_df.assign(chapter = line_is_chapter_harrypotter(harrypotter_df))
harrypotter_df

,content,line,chapter
0,,0,0
1,,1,0
2,,2,0
3,,3,0
4,HP 4 - Harry Potter and The Goblet of Fire,4,0
...,...,...,...
6764,"Har­ry winked at them, turned to Un­cle...",6764,37
6765,"As Ha­grid had said, what would come, w...",6765,37
6766,,6766,37
6767,,6767,37


In [12]:
def line_is_chapter_davinci(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("Prologue|Epilogue|CHAPTER [\d]+|HOUSE", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
davinci_df = davinci_df.assign(chapter = line_is_chapter_davinci(davinci_df))
davinci_df

,content,line,chapter
0,,0,0
1,The Da Vinci Code,1,0
2,Dan Brown,2,0
3,,3,0
4,FOR BLYTHE... AGAIN. MORE THAN EVER.,4,0
...,...,...,...
5887,,5887,109
5888,,5888,109
5889,,5889,109
5890,,5890,109


In [13]:
def line_is_chapter_clouds(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("^Prologue|Chapter [\d]+|About the Author", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
clouds_df = clouds_df.assign(chapter = line_is_chapter_clouds(clouds_df))
clouds_df = clouds_df[0:257]
clouds_df

,content,line,chapter
0,,0,0
1,Head,1,0
2,IN THE,2,0
3,Clouds,3,0
4,,4,0
...,...,...,...
252,,252,44
253,Chapter 44 O NE MONTH LATER China cups clink...,253,45
254,,254,45
255,,255,45


In [14]:
def line_is_chapter_cosmos(dataframe):
    chapter_list = []
    curr_chapter = 0
    for index, row in dataframe.iterrows():
        if re.search("CHAPTER [IVX]+|Chapter V", row['content']):
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

curr_chapter = 0
cosmos_df = cosmos_df.assign(chapter = line_is_chapter_cosmos(cosmos_df))
cosmos_df

,content,line,chapter
0,ACKNOWLEDGMENTS,0,0
1,"Besides those thanked in the introduction, I ...",1,0
2,,2,0
3,By Carl Sagan Published by The Random House P...,3,0
4,,4,0
...,...,...,...
1703,,1703,13
1704,,1704,13
1705,,1705,13
1706,,1706,13
